In [11]:
import os
import pandas as pd
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


In [12]:
!pip install langchain-core

In [ ]:
os.environ["OPENAI_API_KEY"] = "."  # ← 수정!

# 파일 및 설정
CSV_FILE = "./entertainment_ver.csv"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "seoul-entertainment"
                                                                                                                                                                                                              
print("✅ 설정 완료!")

✅ 설정 완료!


In [14]:
import pandas as pd
from langchain_core.documents import Document
# CSV_FILE 변수가 정의되어 있다고 가정합니다.
# df 변수에 데이터프레임이 로드되어야 합니다.

print("📂 CSV 파일 로딩 및 전처리 중...")

# CSV 파일 로드 (이 코드는 이미 실행되었다고 가정하고, df 변수를 사용합니다.)
df = pd.read_csv(CSV_FILE, encoding="utf-8") 

# 이전에 확인된 컬럼 목록:
# ['entertainment_id', 'title', 'url', 'description', 'address', 'transportation', 
#  'image_urls', 'image_count', 'Hours of Operation', 'Important', 'Fee', 'latitude', 'longitude']

print(f"✅ 총 {len(df)}개 항목 로드")
print(f"\n컬럼: {df.columns.tolist()}")

# 의미 있는 Document 생성
docs = []
for idx, row in df.iterrows():
    # 1. 임베딩할 텍스트 (page_content) 생성 - 검색 성능 향상을 위해 중요한 정보만 조합
    text = f"""
제목: {row['title']}
주소: {row['address']}
설명: {row['description']}
    """.strip()
    
    # 2. Document 생성
    doc = Document(
        page_content=text,
        metadata={
            # ID 및 검색/필터링에 자주 쓰일 컬럼
            'entertainment_id': int(row['entertainment_id']),
            'title': str(row['title']),
            'address': str(row['address']),
            'latitude': float(row['latitude']),
            'longitude': float(row['longitude']),
            
            # 모든 나머지 컬럼 (Payload에 저장됨)
            'url': str(row['url']),
            'description': str(row['description']),
            'transportation': str(row['transportation']),
            'image_urls': str(row['image_urls']),
            'image_count': int(row['image_count']),
            'Hours of Operation': str(row['Hours of Operation']),
            'Important': str(row['Important']),
            'Fee': str(row['Fee'])
        }
    )
    docs.append(doc)

print(f"✅ {len(docs)}개 Document 생성 완료!")
print(f"\n첫 번째 항목 샘플:")
print(docs[0].page_content)

📂 CSV 파일 로딩 및 전처리 중...
✅ 총 229개 항목 로드

컬럼: ['entertainment_id', 'title', 'url', 'description', 'address', 'transportation', 'image_urls', 'image_count', 'Hours of Operation', 'Important', 'Fee', 'latitude', 'longitude']
✅ 229개 Document 생성 완료!

첫 번째 항목 샘플:
제목: YG Entertainment Headquarters
주소: 7 Huiujeong-ro 1-gil, Mapo-gu, Seoul
설명: YG Entertainment is one of Korea's major entertainment companies. The roster of celebrities that are under YG Entertainment is huge, and celebrities such as WINNER, iKon, BLACKPINK, and BABYMONSTER are all part of YG. The YG building is notable for itsunique and refined architectural design. Located in Hapjeong-dong, Mapo-gu, visitors will find it easiest to travel to Hapjeong Station to get to the YG building.


In [15]:
print("🤖 OpenAI 임베딩 모델 준비 중...")

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

print("✅ 임베딩 모델 준비 완료!")

🤖 OpenAI 임베딩 모델 준비 중...
✅ 임베딩 모델 준비 완료!


In [16]:
# 셀 5: Qdrant 연결 및 컬렉션 생성
print("🔌 Qdrant 연결 중...")

qdrant_client = QdrantClient(url=QDRANT_URL)

# 기존 컬렉션 삭제
try:
    qdrant_client.delete_collection(collection_name=COLLECTION_NAME)
    print("🗑️  기존 컬렉션 삭제됨")
except:
    print("ℹ️  기존 컬렉션 없음")

# 새 컬렉션 생성
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

print(f"✅ 컬렉션 '{COLLECTION_NAME}' 생성 완료!")

🔌 Qdrant 연결 중...
🗑️  기존 컬렉션 삭제됨
✅ 컬렉션 'seoul-entertainment' 생성 완료!


In [17]:
# 셀 6: 데이터 임베딩 및 저장
print(f"\n💾 {len(docs)}개 레스토랑 임베딩 생성 및 저장 중...")
print("⏳ OpenAI API 호출 중... (약 10-15분 소요)")

vector_store = Qdrant.from_documents(
    documents=docs,
    embedding=embedding,
    url=QDRANT_URL,
    collection_name=COLLECTION_NAME,
)

print(f"\n✅ 총 {len(docs)}개 레스토랑 저장 완료!")
print(f"📊 Qdrant UI: {QDRANT_URL}/dashboard")


💾 229개 레스토랑 임베딩 생성 및 저장 중...
⏳ OpenAI API 호출 중... (약 10-15분 소요)

✅ 총 229개 레스토랑 저장 완료!
📊 Qdrant UI: http://localhost:6333/dashboard


In [18]:
# 셀 7: 저장 확인
collection_info = qdrant_client.get_collection(COLLECTION_NAME)

print(f"📊 컬렉션 정보:")
print(f"  - 컬렉션명: {COLLECTION_NAME}")
print(f"  - 저장된 레코드: {collection_info.points_count}개")
print(f"  - 벡터 차원: {collection_info.config.params.vectors.size}")

📊 컬렉션 정보:
  - 컬렉션명: seoul-entertainment
  - 저장된 레코드: 229개
  - 벡터 차원: 1536


In [ ]:
# 셀 8: 검색 함수 정의 (수정됨)
def search_entertainment(query, k=5):
    """엔터테인먼트 검색"""
    print(f"🔍 질의: '{query}'")
    print("=" * 60)
    
    # vector_store는 Qdrant와 연결된 LangChain 벡터 스토어 객체라고 가정합니다.
    results = vector_store.similarity_search(query, k=k)
    
    for i, doc in enumerate(results, 1):
        print(f"\n[{i}위]")
        print("-" * 60)
        
        # 메타데이터 출력 (새로운 컬럼 이름 사용)
        meta = doc.metadata
        print(f"🆔 ID: {meta.get('entertainment_id', 'N/A')}")
        print(f"🎬 제목: {meta.get('title', 'N/A')}")
        print(f"📍 주소: {meta.get('address', 'N/A')}")
        print(f"⏰ 운영: {meta.get('Hours of Operation', 'N/A')}")
        print(f"ℹ️ 중요: {meta.get('Important', 'N/A')}")
        
        print(f"\n내용:")
        # page_content에는 title, address, description, transportation, Hours of Operation, Fee 등이 포함되어 있습니다.
        print(doc.page_content[:200].replace('\n', ' ') + "...")
        print("-" * 60)
    
    return results

print("✅ 검색 함수 준비 완료!")

✅ 검색 함수 준비 완료!


In [20]:
search_entertainment("kpop", k=3)

🔍 질의: 'kpop'

[1위]
------------------------------------------------------------
🆔  ID: 137
🎬 제목: World K-POP Center
📍 주소: 04605  72, Jangchungdan-ro, Jung-gu, Seoul (World Kpop Center)
⏰ 운영: 10:00-19:00 KST (90min class)
ℹ️ 중요: The World K-POP Center: The Holy Land of K-POP Auditions

The expansive training center at the World K-POP Center, spanning over 1652 square meters, has been the venue for numerous K-POP auditions. This is where it all happened, from domestic shows like the 'Produce 101' series to global K-POP audition programs in China, Japan, Southeast Asia, Europe, and beyond.

Currently, a project is underway to select national representative girl groups from 32 countries around the world. The finalists will receive production and systematic training at the World K-POP Center Training Center here, ultimately leading to their debut.

내용:
제목: World K-POP Center 주소: 04605  72, Jangchungdan-ro, Jung-gu, Seoul (World Kpop Center) 설명: Being the first and largest K-POP institution 

[Document(metadata={'entertainment_id': 137, 'title': 'World K-POP Center', 'address': '04605\xa0\xa072, Jangchungdan-ro, Jung-gu, Seoul (World Kpop Center)', 'latitude': 37.5515434, 'longitude': 127.0025388, 'url': 'https://english.visitseoul.net/entertainment/ShortFormDanceClass/ENPwdn7lf', 'description': 'Being the first and largest K-POP institution in South Korea, the World K-POP Center has dedicated significant efforts to building the education system and nurturing talent in the K-POP industry, starting from its early days as a Hallyu training center. Currently, it offers one-day K-POP dance classes that are easy and fun for anyone to join. These classes provide a unique learning experience within a short timeframe, featuring the latest trendy K-POP songs and high-quality videos.', 'transportation': 'Subway Line 3, Dongguk Univ. Station (Jangchung-dong), Exit 6 -> Bus No. 01\nSubway Lines 2/4/5, Dongdaemun History & Culture Park Station, Exit 9 -> Bus No. 420', 'image_urls': 'htt